In [2]:
from framework.Platform import PlatformManager
from framework.Configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True,
    prc=True,
    ppl=False,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

#platform_manager.calc_metrics()



[INFO]: Real images source found, name:CelebA_Original
[INFO]: 4 different generator sources found. Names:
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP


In [1]:
from framework.Platform import PlatformManager
from framework.Configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True,
    prc=True,
    ppl=False,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   generated_images_path=os.path.join(os.getcwd(), "tmp"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()

[INFO]: Real images source found, name:CelebA_Original
[INFO]: 1 different generator sources found. Names:
original_64
[INFO]: Comparison real-to-real (True)
[START]: Calculating Metrics for original_64
[INFO]: Start Calculation IS, Source = original_64


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased']
Extracting features from input1
Processing samples                                                               
Inception Score: 3.5856323974726556 ± 0.020385455507128
Creating feature extractor "inception-v3-compat" with features ['2048']


[INFO]: IS finished
[INFO]: Start Calculation FID, Source = original_64


Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                               
Frechet Inception Distance: -5.684341886080802e-14
Kernel Inception Distance: -1.4059582082075472e-05 ± 9.778459226345613e-05      


[INFO]: FID finished
[INFO]: Start Calculation KID, Source = original_64
[INFO]: KID finished
[INFO]: Start Calculation Improved PRC, Source = original_64


...

Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                               
Frechet Inception Distance: -5.684341886080802e-14
Kernel Inception Distance: -1.4059582082075472e-05 ± 9.778459226345613e-05      


[INFO]: FID finished
[INFO]: Start Calculation KID, Source = CelebA_Original
[INFO]: KID finished
[INFO]: Start Calculation Improved PRC, Source = CelebA_Original


Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1
Processing samples                                                             
Extracting features from input2
Processing samples                                                             
Precision: 0.7380800247192383
Recall: 0.7356799840927124
F-score: 0.7368780501546134


[INFO]: Improved PRC finished
[FINISHED]: Calculating Metrics for CelebA_Original
{'Inception Score Mean': 3.5856323974726556, 'Inception Score Std': 0.020385455507128, 'Frechet Inception Distance': -5.684341886080802e-14, 'Kernel Inception Distance Mean': -1.4059582082075472e-05, 'Kernel Inception Distance Std': 9.778459226345613e-05, 'Precision': 0.7380800247192383, 'Recall': 0.7356799840927124, 'F1 Score': 0.7368780501546134}


ValueError: too many values to unpack (expected 2)

In [12]:
import numpy as np

for generator in result_dict.data:
    print("------------------------------------")
    print(f"Generator Name: {generator}")
    print("------------------------------------")
    for metric in result_dict.data[generator]:
        print(f"{metric}: {np.round(result_dict.data[generator][metric], 3)}")

------------------------------------
Generator Name: original_64
------------------------------------
Inception Score Mean: 3.586
Inception Score Std: 0.02
Frechet Inception Distance: -0.0
Kernel Inception Distance Mean: -0.0
Kernel Inception Distance Std: 0.0
Precision: 0.739
Recall: 0.735
F1 Score: 0.737
------------------------------------
Generator Name: CelebA_Original
------------------------------------
Inception Score Mean: 3.586
Inception Score Std: 0.02
Frechet Inception Distance: -0.0
Kernel Inception Distance Mean: -0.0
Kernel Inception Distance Std: 0.0
Precision: 0.738
Recall: 0.736
F1 Score: 0.737


In [5]:
import torch_fidelity
import torch

print(torch.cuda.is_available())

metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("guidedDiffusion_IP").get_dataset(), 
    cuda=True, 
    isc=False, 
    fid=False, 
    kid=False, 
    prc=False, 
    verbose=True,
    save_cpu_ram=False,
)

True


ValueError: At least one metric must be specified

In [4]:
metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("diffusionStyleGAN2").get_dataset(), 
    cuda=True, 
    isc=False, 
    fid=False, 
    kid=False, 
    prc=True, 
    verbose=True,
    save_cpu_ram=False,
)

Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1
Processing samples                                                             
Extracting features from input2
Processing samples                                                             
Precision: 0.6168400049209595
Recall: 0.3585200011730194
F-score: 0.4534725186722886


In [3]:
from framework.util import img_to_64

img_to_64("D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\train\\train\\",
             "D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\64res\\")

(50000, 64, 64, 3) size array saved into celeba64_train.npz


In [ ]:
from 3rd